## Step 1: Install Necessary Libraries

In [ ]:
!nvidia-smi

Sat Mar 29 07:34:38 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   76C    P0             43W /   70W |    6512MiB /  15360MiB |     52%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install datasets
!pip install transformers

In [ ]:
import pandas as pd
import warnings
warnings.simplefilter("ignore")

# Step 2: Load and Prepare the Dataset

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving cyberbullying_tweets.csv to cyberbullying_tweets (1).csv


In [ ]:
import pandas as pd
import io

df = pd.read_csv(io.BytesIO(uploaded['cyberbullying_tweets (1).csv']))
print(df)

                                              tweet_text cyberbullying_type
0      In other words #katandandre, your food was cra...  not_cyberbullying
1      Why is #aussietv so white? #MKR #theblock #ImA...  not_cyberbullying
2      @XochitlSuckkks a classy whore? Or more red ve...  not_cyberbullying
3      @Jason_Gio meh. :P  thanks for the heads up, b...  not_cyberbullying
4      @RudhoeEnglish This is an ISIS account pretend...  not_cyberbullying
...                                                  ...                ...
47687  Black ppl aren't expected to do anything, depe...          ethnicity
47688  Turner did not withhold his disappointment. Tu...          ethnicity
47689  I swear to God. This dumb nigger bitch. I have...          ethnicity
47690  Yea fuck you RT @therealexel: IF YOURE A NIGGE...          ethnicity
47691  Bro. U gotta chill RT @CHILLShrammy: Dog FUCK ...          ethnicity

[47692 rows x 2 columns]


In [ ]:
dataset = df

In [ ]:
#from datasets import load_dataset
#dataset = load_dataset('df')

In [ ]:
dataset

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying
...,...,...
47687,"Black ppl aren't expected to do anything, depe...",ethnicity
47688,Turner did not withhold his disappointment. Tu...,ethnicity
47689,I swear to God. This dumb nigger bitch. I have...,ethnicity
47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,ethnicity


## Step 3: Preprocess the Data
Tokenize the dataset using the tokenizer associated with the pre-trained model.

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset  # Ensure you're using Hugging Face's dataset

# Load the dataset (Example: IMDB reviews)
dataset = load_dataset("imdb")

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Tokenize function with batched parameter
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

# Apply tokenization
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Check the tokenized dataset
print(tokenized_datasets)


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})


In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [ ]:
tokenized_datasets["train"][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

## Step 4: Set Up the Training Arguments
Specify the hyperparameters and training settings.

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    eval_strategy ="epoch",     # Evaluate every epoch
    learning_rate=2e-5,              # Learning rate
    per_device_train_batch_size=16,  # Batch size for training
    per_device_eval_batch_size=16,   # Batch size for evaluation
    num_train_epochs=1,              # Number of training epochs
    weight_decay=0.01,               # Strength of weight decay
)
training_args

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=IntervalStrategy.EPOCH,
eval_

## Step 5: Initialize the Model
Load the pre-trained model and define the training procedure.

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer

# Load the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test']
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Step 6: Train the Model
Fine-tune the pre-trained model on your specific dataset.

In [30]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,0.199000,0.177552


TrainOutput(global_step=1563, training_loss=0.2514750437330757, metrics={'train_runtime': 3234.8628, 'train_samples_per_second': 7.728, 'train_steps_per_second': 0.483, 'total_flos': 6577776384000000.0, 'train_loss': 0.2514750437330757, 'epoch': 1.0})

## Step 7: Evaluate the Model
Assess the model's performance on a validation set.

In [31]:
# Evaluate the model
results = trainer.evaluate()
print(results)

{'eval_loss': 0.17755213379859924, 'eval_runtime': 777.2561, 'eval_samples_per_second': 32.164, 'eval_steps_per_second': 2.011, 'epoch': 1.0}


## Step 8: Save the Fine-Tuned Model
Save the fine-tuned model for later use.

In [32]:
# Save the model
model.save_pretrained('./fine-tuned-model')
tokenizer.save_pretrained('./fine-tuned-tokenizer')


('./fine-tuned-tokenizer/tokenizer_config.json',
 './fine-tuned-tokenizer/special_tokens_map.json',
 './fine-tuned-tokenizer/vocab.txt',
 './fine-tuned-tokenizer/added_tokens.json',
 './fine-tuned-tokenizer/tokenizer.json')

In [33]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the saved model and tokenizer
model_path = "./fine-tuned-model"  # Update if saved elsewhere
tokenizer_path = "./fine-tuned-tokenizer"

model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

# Set the model to evaluation mode
model.eval()

def predict_sentence(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=-1).item()
    return prediction  # Adjust based on your label mapping

# Example
sentence = "You are an amazing person!"
prediction = predict_sentence(sentence)

print(f"Sentence: {sentence}")
print(f"Prediction: {prediction}")  # Interpret the label according to your dataset


Sentence: You are an amazing person!
Prediction: 1


In [34]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Define label mapping
label_map = {
    0: "not_cyberbullying",
    1: "gender",
    2: "religion",
    3: "other_cyberbullying",
    4: "age",
    5: "ethnicity"
}

# Load the saved model and tokenizer
model_path = "./fine-tuned-model"  # Update if saved elsewhere
tokenizer_path = "./fine-tuned-tokenizer"

model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

# Set the model to evaluation mode
model.eval()

def predict_sentence(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=-1).item()
    return label_map[prediction]  # Convert index to category

# Example
sentence = "You are an old fool!"
prediction = predict_sentence(sentence)

print(f"Sentence: {sentence}")
print(f"Prediction: {prediction}")


Sentence: You are an old fool!
Prediction: not_cyberbullying


In [36]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Define label mapping
label_map = {
    0: "not_cyberbullying",
    1: "gender",
    2: "religion",
    3: "other_cyberbullying",
    4: "age",
    5: "ethnicity"
}

# Load the saved model and tokenizer
model_path = "./fine-tuned-model"  # Update if saved elsewhere
tokenizer_path = "./fine-tuned-tokenizer"

model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

# Set the model to evaluation mode
model.eval()

def predict_sentence(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=-1).item()
    return label_map[prediction]  # Convert index to category

# Example
sentence = "I love gay and rape jokes hahahaha!!!! especially in the locker room hahahaha!!!!! why are people such assholes???? hahahahhahahah!!!!!"
prediction = predict_sentence(sentence)

print(f"Sentence: {sentence}")
print(f"Prediction: {prediction}")


Sentence: I love gay and rape jokes hahahaha!!!! especially in the locker room hahahaha!!!!! why are people such assholes???? hahahahhahahah!!!!!
Prediction: gender


In [38]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Define label mapping
label_map = {
    0: "not_cyberbullying",
    1: "gender",
    2: "religion",
    3: "other_cyberbullying",
    4: "age",
    5: "ethnicity"
}

# Load the saved model and tokenizer
model_path = "./fine-tuned-model"  # Update if saved elsewhere
tokenizer_path = "./fine-tuned-tokenizer"

model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

# Set the model to evaluation mode
model.eval()

def predict_sentence(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=-1).item()
    return label_map[prediction]  # Convert index to category

# Example
sentence = "she said cause u black (colored) , gay , and u wear fake hair , when did i become gay? and this racism has got to stop"
prediction = predict_sentence(sentence)

print(f"Sentence: {sentence}")
print(f"Prediction: {prediction}")


Sentence: she said cause u black (colored) , gay , and u wear fake hair , when did i become gay? and this racism has got to stop
Prediction: not_cyberbullying


In [40]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Define label mapping
label_map = {
    0: "not_cyberbullying",
    1: "gender",
    2: "religion",
    3: "other_cyberbullying",
    4: "age",
    5: "ethnicity"
}

# Load the saved model and tokenizer
model_path = "./fine-tuned-model"  # Update if saved elsewhere
tokenizer_path = "./fine-tuned-tokenizer"

model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

# Set the model to evaluation mode
model.eval()

def predict_sentence(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=-1).item()
    return label_map[prediction]  # Convert index to category

# Example
sentence = "@alikoeln He is following the example of his prophet who married one of his wives at 6. They do everything their prophet does exactly. #Isam"
prediction = predict_sentence(sentence)

print(f"Sentence: {sentence}")
print(f"Prediction: {prediction}")

Sentence: @alikoeln He is following the example of his prophet who married one of his wives at 6. They do everything their prophet does exactly. #Isam
Prediction: gender
